# TikTokers (TikTok Hackathon 2023)
## Problem Set 1: Optimise Advertisement Moderation

## Import Libraries

In [5]:
import pandas as pd
from ad_scorer import compute_risk_scores
from preprocessing import clean_ads
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from simulator import simulate

## Preprocessing step

In [6]:

path = "../data/data.xlsx"
ads_table = pd.read_excel(path, sheet_name = 0, header = 1)
mod_table = pd.read_excel(path, sheet_name = 1)

ads_table = clean_ads(ads_table)

ads_table.describe()

/Users/kevin/Documents/GitHub/TikTokers/env/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/kevin/Documents/GitHub/TikTokers/src/preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads['punish_num'] = ads['punish_num'].fillna(mode_by_group)
/Users/kevin/Documents/GitHub/TikTokers/src/preprocessing.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,p_date,ad_id,punish_num,ad_revenue,avg_ad_revenue,baseline_st,days_since_last_penalty,days_since_start_time,revenue_ratio
count,28119.0,2.811900e+04,28119.000000,28119.000000,28119.000000,28119.000000,28119.000000,28119.000000,28119.000000
mean,20230807.0,1.772808e+15,1.009566,78.868235,27.205505,1.273209,8166.264056,7.256410,7.225628
std,0.0,2.791198e+12,0.218338,505.321665,173.458473,0.629597,1919.437127,30.750441,266.623935
min,20230807.0,1.714641e+15,1.000000,0.000100,0.000100,0.540000,0.000000,-1.000000,0.000005
25%,20230807.0,1.773293e+15,1.000000,1.810000,1.256850,0.540000,8619.000000,-1.000000,0.861865
50%,20230807.0,1.773511e+15,1.000000,6.680000,3.391200,1.530000,8619.000000,0.000000,1.869176
75%,20230807.0,1.773558e+15,1.000000,21.633250,10.808000,1.780000,8619.000000,2.000000,3.978838
max,20230807.0,1.773616e+15,16.000000,27100.620000,17144.498600,7.590000,8619.000000,649.000000,43645.985507


## Scale features

In [ ]:
scaler_ads = StandardScaler()
X_ads = scaler_ads.fit_transform(
    ads_table.drop(columns=['ad_id', 'delivery_country', 'queue_market', 'days_since_start_time']).values)

scaler_mods = StandardScaler()
X_mods = scaler_mods.fit_transform(mods_table.values)

## Define the Loss Function

In [ ]:
def loss(ads: pd.DataFrame, mods: pd.DataFrame):
    # Loss is determined by combination of results obtained from the simulation.
    utilization, mismatch = simulate(ads, mods)
    return

## Run the Simulation
Monte Carlo simulation of the advertisement allocation process. From the simulations we will take the pair of models that arrived at the best possible result based on our loss function.

In [4]:
max_iterations = 10000
best_loss = float('inf')  # or -float('inf') if you're maximizing
best_gmms = None # will be a tuple of the best models for ads and moderators after running all simulations
tolerance = 1e-5
n_components = 3 # use if using GaussianMixture instead of BayesianGaussianMixture

previous_loss = best_loss

for i in range(max_iterations):
    gmm_ads = GaussianMixture(n_components=n_components)
    gmm_ads.fit(X_ads)

    gmm_mods = GaussianMixture(n_components=n_components)
    gmm_mods.fit(X_mods)

    ad_risk_scores = compute_risk_scores(X_ads, gmm_ads)
    mod_scores = compute_risk_scores(X_mods, gmm_mods)
    
    X_ads['score'] = ad_risk_scores
    X_mods['score'] = mod_scores
    current_loss = loss(X_ads, X_mods) # run simulation based on the dataframe of ads and moderators with the assigned scores
    print(f"Iteration: {i}    Loss: {current_loss}")

    # if abs(current_loss - previous_loss) < tolerance:
    #     break

    if current_loss < best_loss:  # Use > if you're maximizing the loss
        best_loss = current_loss
        best_gmms = (gmm_ads, gmm_mods)

    previous_loss = current_loss

# best_gmms now contains the models with the lowest observed loss
print(f"Best GMM models achieved loss of: {best_loss}")

NameError: name 'X_ads' is not defined